In [1]:
import pandas as pd
import pyspark
import numpy as np
from pydataset import data
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [31]:
# Read the case, department, and source data into their own spark dataframes.

source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

cases = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

source.show(5)
cases.show(3, vertical = True)
dept.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1---------------

In [3]:
# Write the code necessary to store the source data in both csv and json format,
# store these as sources_csv and sources_json

source.write.json("source_json", mode="overwrite")
source.write.csv("source_csv", mode="overwrite")

Write the code necessary to cast the values to the appropriate types.

In [4]:
# Appropriate

source.describe()

DataFrame[summary: string, source_id: string, source_username: string]

In [5]:
# Fixing is needed, case_id doesn't need to be an integer, case_late can be a bool, case_closed can be a bool,
# council district can be a string

cases.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [6]:
cases = cases.withColumn("case_closed", cases.case_closed == "YES").withColumn("case_late", expr('case_late=="YES"'))
cases = cases.withColumn("case_id", cases.case_id.cast('string')).withColumn("council_district", cases.council_district.cast('string'))
cases.show(vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | false                
 num_days_late        | -2.0126041

In [7]:
# Last column could be a bool

dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [8]:
dept = dept.withColumn("dept_subject_to_SLA", expr('dept_subject_to_SLA=="YES"'))
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



In [9]:
# How old is the latest (in terms of days past SLA) currently open issue?

open_cases = cases.where(cases["case_closed"] == False)
open_cases.sort(desc('SLA_days')).show(1, vertical = True)

# How long has the oldest (in terms of days since opened) currently opened issue been open?

open_cases.sort(desc('num_days_late')).show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1013896575           
 case_opened_date     | 9/22/17 8:27         
 case_closed_date     | null                 
 SLA_due_date         | 8/11/21 8:30         
 case_late            | false                
 num_days_late        | -1318.354167         
 case_closed          | false                
 dept_division        | Signals              
 service_request_type | Signal Timing Mod... 
 SLA_days             | 1419.00191           
 case_status          | Open                 
 source_id            | CRM_Listener         
 request_address      | 4200  HARRY WURZB... 
 council_district     | 10                   
only showing top 1 row

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 1/1/17 13:48         
 case_closed_date     | null                 
 SLA_due_date         | 1/17/17 8:30         
 case_late            | true                 
 num_days_

In [10]:
# How many Stray Animal cases are there?

cases.where(cases["service_request_type"] == 'Stray Animal').groupBy(['service_request_type']).count().show()

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+



In [11]:
# How many service requests that are assigned to the Field Operations department (dept_division) are not 
# classified as "Officer Standby" request type (service_request_type)?

field_op = cases.where(cases.dept_division == 'Field Operations')
field_op.where(field_op.service_request_type != 'Officer Standby').groupBy('dept_division').count().show()

+----------------+------+
|   dept_division| count|
+----------------+------+
|Field Operations|113902|
+----------------+------+



In [12]:
# Convert the council_district column to a string column.

cases.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [27]:
# Extract the year from the case_closed_date column.

cases.select(
    "case_closed_date",
    regexp_extract("case_closed_date", r"\d. ", 0).alias("close_year")
).show(truncate=False)

+----------------+----------+
|case_closed_date|close_year|
+----------------+----------+
|1/1/18 12:29    |18        |
|1/3/18 8:11     |18        |
|1/2/18 7:57     |18        |
|1/2/18 8:13     |18        |
|1/1/18 13:29    |18        |
|1/1/18 14:38    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:33    |18        |
|1/2/18 15:32    |18        |
|1/2/18 15:33    |18        |
|1/2/18 15:33    |18        |
|1/2/18 15:33    |18        |
|1/2/18 15:33    |18        |
|1/2/18 15:33    |18        |
|1/2/18 15:33    |18        |
+----------------+----------+
only showing top 20 rows



In [30]:
# Convert num_days_late from days to hours in new columns num_hours_late.

cases.withColumn("num_hours_late", cases.num_days_late*24).show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 num_hours_late       | -23964.2102784       
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | false     

In [33]:
# Join the case data with the source and department data.

cases.join(dept, on=cases.dept_division == dept.dept_division).show(vertical = True)

-RECORD 0--------------------------------------
 case_id                | 1014127332           
 case_opened_date       | 1/1/18 0:42          
 case_closed_date       | 1/1/18 12:29         
 SLA_due_date           | 9/26/20 0:42         
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 dept_division          | Field Operations     
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 source_id              | svcCRMLS             
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 dept_division          | Field Operations     
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 case_id                | 1014127333    

In [ ]:
# Are there any cases that do not have a request source?

